# BM25: Best Matching 25

It is a ranking function used by search engines to estimate the relevance of documents to a given search query. It is a modified version of TF-IDF, taking into consideration term frequency saturation and document length normalization.

## Intuition

- **TF-IDF** gives higher scores to documents containing more occurrences of the query terms, but it does not take into consideration for diminishing returns (i.e., after a certain point, more occurrences of a word do not add much value).
- **BM25** solves this by introducing parameters that control the affect of term frequency and document length.


**Inverse Document Frequency (IDF):**

$$
\text{IDF}(q_i) = \log\left(\frac{N - n(q_i) + 0.5}{n(q_i) + 0.5}\right)
$$

Where:
- $N$ = total number of documents
- $n(q_i)$ = number of documents containing term $q_i$


**BM25 Score:**

$$
\text{BM25}(D, Q) = \sum_{q_i \in Q} \text{IDF}(q_i) \cdot \frac{f(q_i, D) \cdot (k_1 + 1)}{f(q_i, D) + k_1 \cdot (1 - b + b \cdot \frac{|D|}{\text{avgdl}})}
$$

Where:
- $f(q_i, D)$ = frequency of $q_i$ in document $D$
- $|D|$ = length of document $D$
- $\text{avgdl}$ = average document length
- $k_1, b$ = parameters (commonly $k_1=1.5$, $b=0.75$)


In [7]:
import math

docs = [
    "the cat sat on the mat",
    "the cat lay on the rug",
    "the dog barked at the cat"
]

query = "cat on mat".split()

## Tokenize Documents and Set Parameters

In [8]:
tokenized_docs = [doc.split() for doc in docs]

k1 = 1.5
b = 0.75
N = len(docs)

doc_lens = [len(doc) for doc in tokenized_docs]
avgdl = sum(doc_lens) / N

## Compute IDF for Each Query Term

In [9]:
def doc_freq(term):
    return sum(1 for doc in tokenized_docs if term in doc)

idf = {}
for term in query:
    n_t = doc_freq(term)
    idf[term] = math.log((N - n_t + 0.5) / (n_t + 0.5) + 1e-10)  

## Term Frequency and BM25 Scoring Functions

In [10]:
def term_freq(term, doc):
    return doc.count(term)

def bm25_score(doc):
    score = 0
    doc_len = len(doc)
    for term in query:
        f = term_freq(term, doc)
        numerator = f * (k1 + 1)
        denominator = f + k1 * (1 - b + b * (doc_len / avgdl))
        score += idf[term] * (numerator / denominator) if denominator != 0 else 0
    return score

## Results

The BM25 scores printed for each document shows how relevant each document is to the query "cat on mat". The document with the highest score is considered the most relevant. BM25 takes into consideration not just the presence of query terms, but also their frequency and the length of each document.

In [11]:
scores = [bm25_score(doc) for doc in tokenized_docs]

for i, score in enumerate(scores):
    print(f"Doc {i+1} BM25 score: {score:.4f}")

Doc 1 BM25 score: -1.9459
Doc 2 BM25 score: -2.4567
Doc 3 BM25 score: -1.9459
